In [28]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
load_dotenv()

DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "botgpt"
DB_PASSWORD = "pass"
DB_NAME = "bot_gpt_test"

In [35]:
def get_server_connection():
    return mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
    )

def get_connection():
    return mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
    )
    
def ensure_database():
    try:
        conn = get_server_connection()
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4;")
        conn.commit()
        cursor.close()
        conn.close()
        print(f"ensured {DB_NAME} database")
    except Error as e:
        print(f"Error ensuring database: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
    
def init_db_test():
    ensure_database()

    try:
        conn = get_connection()
        cursor = conn.cursor()
    
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sessions (
                id INT AUTO_INCREMENT PRIMARY KEY,
                session_id VARCHAR(255) NOT NULL UNIQUE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)

        print("done")

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INT AUTO_INCREMENT PRIMARY KEY,
                session_id VARCHAR(255) NOT NULL,
                role VARCHAR(50) NOT NULL,
                content TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (session_id) REFERENCES sessions(session_id)
                    ON DELETE CASCADE
            );
        """)

        conn.commit()
        cursor.close()
        conn.close()
        print("Database tables created")
    except Error as e:
        print(f"Error creating tables: {e}")

In [36]:
init_db_test()

ensured bot_gpt_test database
done
Database tables created


In [37]:
def add_session(session_id: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT IGNORE INTO sessions (session_id) VALUES (%s)", (session_id,))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"[OK] Added session: {session_id}")
    except Error as e:
        print("[ERR] add_session:", e)


def list_sessions():
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT id, session_id, created_at FROM sessions")
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        return rows
    except Error as e:
        print("[ERR] list_sessions:", e)
        return []


In [38]:
def add_message(session_id: str, role: str, content: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO messages (session_id, role, content) VALUES (%s, %s, %s)",
            (session_id, role, content)
        )
        conn.commit()
        cursor.close()
        conn.close()
        print(f"[OK] Added message to {session_id} ({role})")
    except Error as e:
        print("[ERR] add_message:", e)


def get_messages(session_id: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "SELECT id, role, content, created_at FROM messages WHERE session_id=%s ORDER BY id ASC",
            (session_id,)
        )
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        return rows
    except Error as e:
        print("[ERR] get_messages:", e)
        return []


In [39]:
# Add sessions
add_session("sess_1_test")
add_session("sess_2_test")

print("Available sessions:")
for s in list_sessions():
    print(s)

# Add messages
add_message("sess_1_test", "user", "hello d")
add_message("sess_1_test", "assistant", "hey im bot yo yo")
add_message("sess_1_test", "user", "nice")

print("\nMessages for sess_1_test:")
for msg in get_messages("sess_1_test"):
    print(msg)


[OK] Added session: sess_1_test
[OK] Added session: sess_2_test
Available sessions:
(1, 'sess_1_test', datetime.datetime(2025, 11, 29, 21, 19, 57))
(2, 'sess_2_test', datetime.datetime(2025, 11, 29, 21, 19, 58))
[OK] Added message to sess_1_test (user)
[OK] Added message to sess_1_test (assistant)
[OK] Added message to sess_1_test (user)

Messages for sess_1_test:
(1, 'user', 'hello d', datetime.datetime(2025, 11, 29, 21, 19, 58))
(2, 'assistant', 'hey im bot yo yo', datetime.datetime(2025, 11, 29, 21, 19, 58))
(3, 'user', 'nice', datetime.datetime(2025, 11, 29, 21, 19, 58))


In [42]:
# testing deletion
def delete_session(session_id: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("DELETE FROM sessions WHERE session_id = %s", (session_id,))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"[OK] Deleted session: {session_id} (and all its messages)")
    except Error as e:
        print("[ERR] delete_session:", e)


In [43]:
delete_session("sess_2_test")
for s in list_sessions():
    print(s)

[OK] Deleted session: sess_2_test (and all its messages)
(1, 'sess_1_test', datetime.datetime(2025, 11, 29, 21, 19, 57))


In [44]:
# deleting messages (we will delete all the later massage from that point)
def delete_from_message(session_id: str, message_id: int):
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # check if message exists
        cursor.execute(
            "SELECT id FROM messages WHERE id = %s AND session_id = %s",
            (message_id, session_id),
        )
        row = cursor.fetchone()
        if not row:
            print(f"[WARN] no message with id={message_id} for session_id={session_id}")
            cursor.close()
            conn.close()
            return

        # delete from this message (and all after it)
        delete_sql = """
            DELETE FROM messages
            WHERE session_id = %s AND id >= %s
        """
        cursor.execute(delete_sql, (session_id, message_id))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"[OK] deleted messages from id={message_id} onwards in session {session_id}")
    except Error as e:
        print("[ERR] delete_from_message:", e)

In [45]:
# testign 

# current messages
test_session = "sess_1_test"
print("Before delete:")
for msg in get_messages(test_session):
    print(msg)

# deleting from 2nd message
# get id of that message:
msgs = get_messages(test_session)
if len(msgs) >= 2:
    cut_id = msgs[1][0]   # msgs[i][0] is 'id' from SELECT
    print(f"\nCutting from message id={cut_id}")
    delete_from_message(test_session, cut_id)

print("\nafter delete:")
for msg in get_messages(test_session):
    print(msg)


Before delete:
(1, 'user', 'hello d', datetime.datetime(2025, 11, 29, 21, 19, 58))
(2, 'assistant', 'hey im bot yo yo', datetime.datetime(2025, 11, 29, 21, 19, 58))
(3, 'user', 'nice', datetime.datetime(2025, 11, 29, 21, 19, 58))

Cutting from message id=2
[OK] deleted messages from id=2 onwards in session sess_1_test

after delete:
(1, 'user', 'hello d', datetime.datetime(2025, 11, 29, 21, 19, 58))
